<a href="https://colab.research.google.com/github/WeAreDobby/CV_project/blob/main/AutoEncoder/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Drive 연동
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython import display

import os
from tqdm import tqdm
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time

In [3]:
# Hyper Parameter
EPOCHS = 10000
channel = 1
image_size = 128
latent_size = 100  #latent vector size
batch_size = 64
learning_rate = 0.0001
read_type = lambda channel: 'rgb' if channel==3 else 'gray'

In [4]:
# Load Pocketmon Dataset
def find_path(path):
    file_list = os.listdir(path)
    return file_list

def png_to_np(path, file_list, read_type='rgb'):
    pix = []
    for idx in tqdm(range(len(file_list))):
      file_name = file_list[idx]
      if read_type == "rgb":
        image = PIL.Image.open(path + '/' + file_name) # (256, 256, 3)
      elif read_type == "gray":
        image = PIL.Image.open(path + '/' + file_name).convert("L") # (256, 256, 1)
      np_image = (np.array(image).astype(np.float32) - 127.5) / 127.5
      pix.append(np_image)
    return pix

In [5]:
path = 'drive/MyDrive/dataset/Human/Human_{}'.format(str(image_size))  # 'drive/MyDrive/dataset/Dobby/Dobby_{}' #'drive/MyDrive/dataset/pockemon/jpg/jpg_dataset_{}'

file_list = find_path(path)
images = png_to_np(path, file_list, read_type=read_type(channel))
train_dataset = np.array(images)

# GrayScale일때 1channel을 만들어주기위해 활용
if channel == 1:
  train_dataset = train_dataset[..., tf.newaxis]
  
train_dataset = (tf.data.Dataset.from_tensor_slices(train_dataset)
                 .shuffle(30).batch(batch_size))

train_dataset

100%|██████████| 770/770 [03:08<00:00,  4.08it/s]


<BatchDataset shapes: (None, 128, 128, 1), types: tf.float32>

In [6]:
"""inputshape = tf.keras.layers.InputLayer
dense = tf.keras.layers.Dense
conv2d = tf.keras.layers.Conv2D
maxpool = tf.keras.layers.MaxPool2D
flatten = tf.keras.layers.Flatten
reshape = tf.keras.layers.Reshape
batchNormalization = tf.keras.layers.BatchNormalization
leakyrelu = tf.keras.layers.LeakyReLU()

def generator():
  model = tf.keras.Sequential(name='Generator')

  model.add(inputshape((latent_size, )))
  model.add(dense(128, activation=leakyrelu))
  model.add(batchNormalization())
    
  model.add(dense(512, activation=leakyrelu))
  model.add(batchNormalization())
      
  model.add(dense(2048, activation=leakyrelu))
  model.add(batchNormalization())
      
  model.add(dense(image_size**2, activation=leakyrelu))
  model.add(batchNormalization())
      
  model.add(reshape((image_size, image_size, 1)))

  g_model = tf.keras.Model(
    inputs=model.get_layer(index=0),
    outputs=model.get_layer(index=-1),
  )

  return g_model

def discriminator():
  model = tf.keras.Sequential(name='Discriminator')

  model.add(inputshape((image_size, image_size, 1)))
  model.add()
  model.add(maxpool((2, 2)))

  model.add()
  model.add(maxpool((2, 2)))

  model.add()
  model.add(maxpool((2, 2)))

  model.add()
  model.add(flatten())

  model.add(dense(1024, activation=leakyrelu))
  model.add(dense(1))

  d_model = tf.keras.Model(
    inputs=model.get_layer(index=0),
    outputs=model.get_layer(index=-1),
  )

  return d_model"""

"inputshape = tf.keras.layers.InputLayer\ndense = tf.keras.layers.Dense\nconv2d = tf.keras.layers.Conv2D\nmaxpool = tf.keras.layers.MaxPool2D\nflatten = tf.keras.layers.Flatten\nreshape = tf.keras.layers.Reshape\nbatchNormalization = tf.keras.layers.BatchNormalization\nleakyrelu = tf.keras.layers.LeakyReLU()\n\ndef generator():\n  model = tf.keras.Sequential(name='Generator')\n\n  model.add(inputshape((latent_size, )))\n  model.add(dense(128, activation=leakyrelu))\n  model.add(batchNormalization())\n    \n  model.add(dense(512, activation=leakyrelu))\n  model.add(batchNormalization())\n      \n  model.add(dense(2048, activation=leakyrelu))\n  model.add(batchNormalization())\n      \n  model.add(dense(image_size**2, activation=leakyrelu))\n  model.add(batchNormalization())\n      \n  model.add(reshape((image_size, image_size, 1)))\n\n  g_model = tf.keras.Model(\n    inputs=model.get_layer(index=0),\n    outputs=model.get_layer(index=-1),\n  )\n\n  return g_model\n\ndef discriminator():

In [7]:
# Generator
class Generator(tf.keras.Model):
  def __init__(self):
    super(Generator, self).__init__()
    dense = tf.keras.layers.Dense
    batchNormalization = tf.keras.layers.BatchNormalization
    reshape = tf.keras.layers.Reshape
    leakyrelu = tf.keras.layers.LeakyReLU()

    self.sequence = list()
    # Block 1
    self.sequence.append(dense(128, activation=leakyrelu))
    self.sequence.append(batchNormalization())
    # Block 2
    self.sequence.append(dense(512, activation=leakyrelu))
    self.sequence.append(batchNormalization())
    # Block 3
    self.sequence.append(dense(2048, activation=leakyrelu))
    self.sequence.append(batchNormalization())
    # Block 4
    self.sequence.append(dense(image_size**2, activation=leakyrelu))
    self.sequence.append(batchNormalization())
    
    self.sequence.append(reshape([image_size, image_size, 1]))
    
  def call(self, z):
    for layer in self.sequence:
      z = layer(z)

    return z

In [8]:
# Discriminator
class Discriminator(tf.keras.Model):
  def __init__(self):
    super(Discriminator, self).__init__()
    conv2d = tf.keras.layers.Conv2D
    maxpool = tf.keras.layers.MaxPool2D
    dense = tf.keras.layers.Dense
    flatten = tf.keras.layers.Flatten
    leakyrelu = tf.keras.layers.LeakyReLU()

    self.sequence = list()
    # Block 1
    self.sequence.append(conv2d(8, kernel_size=(3, 3), activation=leakyrelu, padding="same"))  
    self.sequence.append(maxpool((2, 2)))                                      
    # Block 2
    self.sequence.append(conv2d(32, kernel_size=(3, 3), activation=leakyrelu, padding="same"))
    self.sequence.append(maxpool((2, 2)))
    # Block 3
    self.sequence.append(conv2d(64, kernel_size=(3, 3), activation=leakyrelu, padding="same"))
    self.sequence.append(maxpool((2, 2)))
    # Block 4
    self.sequence.append(conv2d(128, kernel_size=(3, 3), activation=leakyrelu, padding="same"))
    self.sequence.append(flatten())
    # Block 5
    self.sequence.append(dense(1024, activation=leakyrelu))
    self.sequence.append(dense(1))
    
  def call(self, x):
    for layer in self.sequence:
      x = layer(x)

    return x

In [9]:
# Noise 생성부
def make_Noise(batch_size, latent_size):
  return tf.random.normal((batch_size, latent_size))

# D_Model Loss 연산
def discriminator_loss(loss, d_result_real, d_result_fake):
    real_loss = loss(tf.ones_like(d_result_real), d_result_real)
    fake_loss = loss(tf.zeros_like(d_result_fake), d_result_fake)
    total_loss = real_loss + fake_loss
    return total_loss

# D_Model Optimizer
def discriminator_optimizer(learning_rate):
    return tf.keras.optimizers.Adam(learning_rate)

# G_Model Loss 연산
def generator_loss(loss, d_result_fake):
    return loss(tf.ones_like(d_result_fake), d_result_fake)

# G_Model Optimizer
def generator_optimizer(learning_rate):
    return tf.keras.optimizers.Adam(learning_rate)

# G_Model에서 생성된 이미지 저장하는거
def generate_and_save_images(g_model, epoch):
  noises = make_Noise(32, 100)
  g_predict = g_model(noises, training=False)

  fig = plt.figure(figsize=(6,6))

  for i in range(g_predict.shape[0]):
      plt.subplot(6, 6, i+1)
      plt.imshow(g_predict[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  #plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

@tf.function
def train_step(g_model, d_model, loss_function, g_optimizer, d_optimizer, batch_size, latent_size, learning_rate, real_images):
  noises = make_Noise(batch_size, latent_size)

  with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
    g_images = g_model(noises)

    d_result_real = d_model(real_images, training=True)
    d_result_fake = d_model(g_images, training=True)

    g_loss = generator_loss(loss_function, d_result_fake)
    d_loss = discriminator_loss(loss_function, d_result_real, d_result_fake)

  g_gradients = g_tape.gradient(g_loss, g_model.trainable_variables)
  d_gradients = d_tape.gradient(d_loss, d_model.trainable_variables)

  g_optimizer.apply_gradients(zip(g_gradients, g_model.trainable_variables))
  d_optimizer.apply_gradients(zip(d_gradients, d_model.trainable_variables))

In [10]:
g_model = Generator()
d_model = Discriminator()

loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)
g_optimizer = generator_optimizer(learning_rate)
d_optimizer = discriminator_optimizer(learning_rate)

for epoch in tqdm(range(EPOCHS)):
  start = time.time()

  for real_images in train_dataset:
    train_step(g_model, d_model, loss_function, g_optimizer, d_optimizer, batch_size, latent_size, learning_rate, real_images)

  if (epoch % 100) == 0:
    generate_and_save_images(g_model, epoch)

  # print (' 에포크 {} 에서 걸린 시간은 {} 초 입니다'.format(epoch +1, time.time()-start))
  # print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

 46%|████▌     | 4576/10000 [33:08<39:16,  2.30it/s]


KeyboardInterrupt: ignored